In [38]:
import pandas as pd
from geocodio import GeocodioClient
from dotenv import load_dotenv
import os

# Load API key from .env file
load_dotenv()
api_key = os.getenv("GEOCODIO_API_KEY")

# Initialize the Geocodio client
client = GeocodioClient(api_key)

# Geocode a single address
def batch_geocode_addresses(addresses):
    try:
        return client.batch_geocode(addresses)
    except Exception as e:
        print(f"Error during batch geocoding: {e}")
        return None

# Load addresses from CSV files
walmart_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/walmart_sample.csv')
zips_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/zips_sample.csv')

# Extract addresses and ZIP codes as lists
walmart_addresses = walmart_df['Address'].tolist()  # Assuming 'Address' is the column name
zip_codes = zips_df['zip'].astype(str).tolist()  # Assuming 'zip' is the column name

# Batch geocode
walmart_geocoded = batch_geocode_addresses(walmart_addresses)
zip_geocoded = batch_geocode_addresses(zip_codes)

In [41]:
# function to extract lat/lng from the geocoded data
def extract_lat_lng(geocoded_data):
    lat_lng_list = []
    for item in geocoded_data:
        if item['results']:  # Check if the results list is not empty
            lat = item['results'][0]['location']['lat']
            lng = item['results'][0]['location']['lng']
            lat_lng_list.append((lat, lng))
        else:
            lat_lng_list.append((None, None))  # Append None values if no results
    return lat_lng_list

walmart_lat_lng = extract_lat_lng(walmart_geocoded)
zip_lat_lng = extract_lat_lng(zip_geocoded)

In [42]:
# Add lat/lng to the Walmart dataframe
walmart_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/walmart_sample.csv')
zip_df = pd.read_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/zips_sample.csv')

# Assuming the length of walmart_lat_lng and zip_lat_lng matches the respective dataframes
walmart_df[['latitude', 'longitude']] = pd.DataFrame(walmart_lat_lng, columns=['latitude', 'longitude'])
zip_df[['latitude', 'longitude']] = pd.DataFrame(zip_lat_lng, columns=['latitude', 'longitude'])

# Save the updated dataframes
walmart_df.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/walmart_sample_geocoded.csv', index=False)
zip_df.to_csv('/Users/pintoza/Desktop/dev/data-science/walmart-proximity/data/interim/zips_sample_geocoded.csv', index=False)